In [1]:
#Library import
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import streamlit as st

import nltk
nltk.download('stopwords')
nltk.download('punkt')

from textblob import TextBlob
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, ENGLISH_STOP_WORDS

from gensim.corpora.dictionary import Dictionary

import spacy

from collections import Counter

import time
import re
import sys
import os
import requests
from pathlib import Path

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/miguel.d.ferrusca/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/miguel.d.ferrusca/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/Users/miguel.d.ferrusca/Documents/ironhack/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
def scraper(asin, pages=10):
    
    df = pd.DataFrame(columns=['rating', 'content', 'title'])
    pages = range(1,pages+1)
    asin=asin
    
    for page in pages:
        
        print('Scraped {} page(s). Scraping page {}. Total reviews: {}'.format(page-1, page, len(df)))
        
        # iterable url
        url = 'https://www.amazon.com/product-reviews/{}/ref=cm_cr_getr_d_paging_btm_prev_1?pageNumber={}'.format(asin, page)
        
        # getting soup
        driver = webdriver.Chrome()
        driver.get(url)
        time.sleep(1)
        page_source = driver.page_source
        driver.quit()
        soup = BeautifulSoup(page_source,'lxml')
        
        # parsing soup
        reviews = soup.findAll("div", {"class":"a-section review aok-relative"})
        ## parsing reviews section
        reviews = BeautifulSoup('<br/>'.join([str(tag) for tag in reviews]), 'html.parser')
        
        #getting title
        title = soup.find("h1", {"class":"a-size-large a-text-ellipsis"}).get_text()
        
        #getting content
        contents = reviews.find_all("span", {"data-hook":"review-body"})
        content_lst = []
        for content in contents:
            text_ = content.find_all("span")[0].get_text("\n").strip()
            text_ = ". ".join(text_.splitlines())
            text_ = re.sub(' +', ' ', text_)
            content_lst.append(text_)
            
        #getting rating
        ratings = reviews.find_all("i", {"data-hook":"review-star-rating"})
        full_rating_lst = []
        for rating in ratings:
            full_rating_lst.append(rating.find_all("span")[0].contents[0])
            
        
        rating_lst = []
        for rating in full_rating_lst:
            rating_lst.append(re.findall("\d+\.\d+", rating)[0])
            
            
        # concatenating to main data frame
        
        temp_df = pd.DataFrame({'rating':rating_lst, 'content':content_lst, 'title':title})
        df = df.append(temp_df)
        
    return df


In [3]:
asins = ['B00GAC1D2G', 'B07RZ74VLR', 
         'B01LZNGPY3', 'B01H6GUCCQ', 
         'B07KXQX3S3', 'B07VGRJDFY', 
         'B07SFKTLZM', 'B071JRMKBH', 
         'B01NAWKYZ0', 'B08FC6C75Y',
         'B01LWVX2RG', 'B01N1037CV', 
         'B08WWFWRY6', 'B07GBZ4Q68', 
         'B01N5OKGLH', 'B08H9M7LDY', 
         'B07SL6ZXBL', 'B08DF248LD', 
         'B010KYDNDG', 'B00NLZUM36',
         'B08CVB3Y8B', 'B07Y693ND1', 
         'B01MY7GHKJ', 'B088GH4X9W',
         'B07F7T8J9P', 'B093B218BN',
         'B07L7W915H', 'B07DHTN9LQ',
         'B08XFPDXDR', 'B07WDGB9P5',
         'B08G9J44ZN', 'B07VGRJDFY',
         'B0931NN4PR', 'B092VT1JGD',
         'B01N6QKT7H', 'B01N6S068R']

asins = np.unique(asins).tolist()

In [4]:
dataset = pd.DataFrame(columns=['rating', 'content', 'title'])

for asin in asins:
    dataset = dataset.append(scraper(asin,5))

Scraped 0 page(s). Scraping page 1. Total reviews: 0
Scraped 1 page(s). Scraping page 2. Total reviews: 10
Scraped 2 page(s). Scraping page 3. Total reviews: 20
Scraped 3 page(s). Scraping page 4. Total reviews: 30
Scraped 4 page(s). Scraping page 5. Total reviews: 40
Scraped 0 page(s). Scraping page 1. Total reviews: 0
Scraped 1 page(s). Scraping page 2. Total reviews: 10
Scraped 2 page(s). Scraping page 3. Total reviews: 20
Scraped 3 page(s). Scraping page 4. Total reviews: 30
Scraped 4 page(s). Scraping page 5. Total reviews: 40
Scraped 0 page(s). Scraping page 1. Total reviews: 0
Scraped 1 page(s). Scraping page 2. Total reviews: 10
Scraped 2 page(s). Scraping page 3. Total reviews: 20
Scraped 3 page(s). Scraping page 4. Total reviews: 30
Scraped 4 page(s). Scraping page 5. Total reviews: 40
Scraped 0 page(s). Scraping page 1. Total reviews: 0
Scraped 1 page(s). Scraping page 2. Total reviews: 10
Scraped 2 page(s). Scraping page 3. Total reviews: 20
Scraped 3 page(s). Scraping page

Scraped 3 page(s). Scraping page 4. Total reviews: 30
Scraped 4 page(s). Scraping page 5. Total reviews: 40
Scraped 0 page(s). Scraping page 1. Total reviews: 0
Scraped 1 page(s). Scraping page 2. Total reviews: 10
Scraped 2 page(s). Scraping page 3. Total reviews: 19
Scraped 3 page(s). Scraping page 4. Total reviews: 19
Scraped 4 page(s). Scraping page 5. Total reviews: 19
Scraped 0 page(s). Scraping page 1. Total reviews: 0
Scraped 1 page(s). Scraping page 2. Total reviews: 10
Scraped 2 page(s). Scraping page 3. Total reviews: 20
Scraped 3 page(s). Scraping page 4. Total reviews: 30
Scraped 4 page(s). Scraping page 5. Total reviews: 40
Scraped 0 page(s). Scraping page 1. Total reviews: 0
Scraped 1 page(s). Scraping page 2. Total reviews: 10
Scraped 2 page(s). Scraping page 3. Total reviews: 20
Scraped 3 page(s). Scraping page 4. Total reviews: 30
Scraped 4 page(s). Scraping page 5. Total reviews: 40
Scraped 0 page(s). Scraping page 1. Total reviews: 0
Scraped 1 page(s). Scraping page

In [5]:
# keeping products with high reviews

reviews_by_product = dataset.groupby(dataset.title)['content'].agg('count').reset_index().sort_values(by='content', ascending=False)
high_reviewed_prods = reviews_by_product[reviews_by_product['content']>=40]['title'].unique().tolist()

clean_df = dataset[dataset['title'].isin(high_reviewed_prods)]

In [6]:
# Getting text polarity and subjectivity

clean_df['polarity'] = [TextBlob(review).sentiment.polarity for review in clean_df['content'].tolist()]
clean_df['subjectivity'] = [TextBlob(review).sentiment.subjectivity for review in clean_df['content'].tolist()]

<ipython-input-6-364de5a1b6c4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['polarity'] = [TextBlob(review).sentiment.polarity for review in clean_df['content'].tolist()]
<ipython-input-6-364de5a1b6c4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['subjectivity'] = [TextBlob(review).sentiment.subjectivity for review in clean_df['content'].tolist()]


In [7]:
# exporting dataset

path = 'data/'

clean_df.to_csv(os.path.join(path,r'dataset.csv'))

In [9]:
clean_df

,rating,content,title,polarity,subjectivity
0,1.0,I purchased two of these cards almost exactly ...,$10 PlayStation Store Gift Card [Digital Code],-0.019792,0.230208
1,5.0,Amazon wants to know what I think about this i...,$10 PlayStation Store Gift Card [Digital Code],0.100676,0.358117
2,5.0,fast easy poor n broke due to fortnite i cant ...,$10 PlayStation Store Gift Card [Digital Code],-0.003496,0.566414
3,5.0,El tiempo de llegada del código es inmediato,$10 PlayStation Store Gift Card [Digital Code],0.000000,0.000000
4,2.0,"This should be a professional service, if you'...",$10 PlayStation Store Gift Card [Digital Code],0.125000,0.462500
...,...,...,...,...,...
5,2.0,I've been a Nintendo fan since the late 80's. ...,Nintendo Joy-Con (L)/(R) Fortnite Fleet Force ...,0.064259,0.456111
6,4.0,"So the joycons works fine, they are original n...",Nintendo Joy-Con (L)/(R) Fortnite Fleet Force ...,0.105087,0.448241
7,1.0,Bought the Switch for Christmas 2017. It's Sep...,Nintendo Joy-Con (L)/(R) Fortnite Fleet Force ...,0.070055,0.329670
8,1.0,I barely played my switch over the last year. ...,Nintendo Joy-Con (L)/(R) Fortnite Fleet Force ...,-0.052222,0.316667
